In [ ]:
# 런타임 에러 -> F12 ->개발자 도구창 -> Console -> 엔터
# function ClickConnect() { 

# var buttons = document.querySelectorAll("colab-dialog.yes-no-dialog paper-button#cancel"); buttons.forEach(function(btn) { btn.click(); });

# console.log("1분마다 자동 재연결"); 

# document.querySelector("colab-toolbar-button#connect").click();

#  }

# setInterval(ClickConnect,1000*60);

In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies


Cloning into 'yolov5'...
remote: Enumerating objects: 9211, done.
remote: Total 9211 (delta 0), reused 0 (delta 0), pack-reused 9211
Receiving objects: 100% (9211/9211), 9.55 MiB | 27.86 MiB/s, done.
Resolving deltas: 100% (6408/6408), done.
/content/yolov5
     |████████████████████████████████| 636 kB 8.3 MB/s 


In [2]:
# 구글 드라이브와 연결하기
# from google.colab import auth
# auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:
!mkdir car_aug
!unzip -uq /content/gdrive/Shareddrives/BigLeader/car_aug.zip -d car_aug

In [6]:
!ls

car_aug		 detect.py   hubconf.py  README.md	   tutorial.ipynb
CONTRIBUTING.md  Dockerfile  LICENSE	 requirements.txt  utils
data		 export.py   models	 train.py	   val.py


In [4]:
#train test set 분리

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

import os

path = "car_aug/images/train/"
img_list_path = os.listdir(path)
img_list = [file for file in img_list_path if file.endswith(".jpg")]
print(img_list)
print("\n\n")

# from glob import glob # 파일 경로명까지 저장 => 폐기

# img_list = glob('Hyundai_0811/images/train/*.jpg')
# print(len(img_list))
# print(img_list)


#train test set 분리
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))
print("\n\n")

# val에 넣고 train에 넣을까

import shutil 

# img파일 train val으로 나누기
for a in val_img_list: 
    filename = a
    src = 'car_aug/images/train/' 
    dir = 'car_aug/images/val/' 
    shutil.move(src + filename, dir + filename)
print(len(val_img_list),val_img_list)
print("\n\n")

# 나뉜 img_val을 기준으로 labeling된 txt 파일  train val 으로 나누기
val_txt_list=[]
for a in val_img_list:
    temp = a.replace("jpg","txt")
    val_txt_list.append(temp)
print(len(val_txt_list),val_txt_list)
print("\n\n")

for a in val_txt_list: 
    filename = a
    src = 'car_aug/labels/train/' 
    dir = 'car_aug/labels/val/' 
    shutil.move(src + filename, dir + filename)

Setup complete. Using torch 1.9.0+cu102 (Tesla P100-PCIE-16GB)
['K_SoulEV(30)_sub30.jpg', 'i30_185_add60.jpg', 'Orando126_CLAHE.jpg', 'Hyundai_Grandeur _IG(34)_add60.jpg', 'Hyundai_YF_Sonata(1)_rotation10.jpg', 'Winstom92_gray.jpg', 'Orando183_rotation350.jpg', 'Winstom55_Gaussian_blur.jpg', 'i30_188_Bilateral_filter.jpg', 'Hyundai_Avante_MD(178)_Bilateral_filter.jpg', 'NFsonata168_CLAHE.jpg', 'Orando119_sub30.jpg', 'Newsorento235_sub60.jpg', 'NFsonata202_Gaussian_blur.jpg', 'Hyundai_Grand_Starex(61)_Gaussian_blur.jpg', 'Hyundai_TheNewKona(7)_rotation10.jpg', 'Hyundai_Grandeur _IG(80)_Bilateral_filter.jpg', 'LFsonata(53)_rotation10.jpg', 'LFsonata(86)_gray.jpg', 'Hyundai_SantaFe_TM(54)_CLAHE.jpg', 'Hyundai_YF_Sonata(13)_rotation350.jpg', 'Hyundai_Grand_Starex(144)_rotation10.jpg', 'Hyundai_SantaFe_TM(151)_gray.jpg', 'Hyundai_Avante_MD(6)_gray.jpg', 'Hyundai_SantaFe_TM(206).jpg', 'Hyundai_Grandeur _HG(236).jpg', 'Orando9_rotation10.jpg', 'Thenewcanival222_noise0.01.jpg', 'Hyundai_Avante

In [5]:
!python train.py --img 416 --batch 16 --epochs 20 --data car.yaml --weights yolov5x.pt --nosave --cache

train: weights=yolov5x.pt, cfg=, data=car.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=20, batch_size=16, imgsz=416, rect=False, resume=False, nosave=True, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=30
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-385-gd7aa3f1 torch 1.9.0+cu102 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.

In [ ]:
# filp, 밝기2개(40,50) 회전4개(10,20,340,350) gray_scale /content/yolov5/car_aug/images/train

In [6]:
import shutil
source = r'/content/yolov5/runs/train/exp'
destination = r'/content/gdrive/MyDrive/Colab Notebooks'
shutil.move(source,destination)

'/content/gdrive/MyDrive/Colab Notebooks/exp'